In [1]:
pip install serpapi

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import time
from urllib.parse import urlparse

def search_business_links(business_name, business_type="general"):
    """Searches for business links based on name and type."""
    api_key = "673154847cda478a5456744c"
    base_url = "https://api.scrapingdog.com/google"
    all_links = []
    seen_domains = set()
    search_queries = [f"{business_name} {suffix}" for suffix in
                      ["", "reviews", f"{business_type}",
                       "food" if business_type == "restaurant" else "services",
                       "accommodations" if business_type == "hotel" else "company"]]

    for query in search_queries:
        for page in range(1, 10):
            params = {
                "api_key": api_key,
                "query": query,
                "num": 50,
                "country": "in",
                "start": (page - 1) * 50
            }
            response = requests.get(base_url, params=params)
            if response.status_code == 200:
                for result in response.json().get("organic_results", []):
                    link = result.get("link")
                    if link and not any(domain in link for domain in ["facebook.com", "instagram.com", "twitter.com", "/aclk?"]):
                        domain = urlparse(link).netloc
                        if domain not in seen_domains:
                            all_links.append(link)
                            seen_domains.add(domain)
            else:
                print(f"Request failed for query '{query}' with status code: {response.status_code}")
            time.sleep(1)  # Be respectful of the API rate limits

    return all_links

def scrape_reviews(url):
    headers = {"User-Agent": "Mozilla/5.0"}  # Customize User-Agent if needed
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return f"Failed to retrieve the page. Status code: {response.status_code}"

    soup = BeautifulSoup(response.text, "html.parser")
    reviews = [element.get_text(strip=True)
               for element in soup.find_all('div', class_='review-container')]  # Adjust class if needed
    return reviews

def get_business_reviews(business_name, business_type="general"):
    links = search_business_links(business_name, business_type)
    all_reviews = {}
    for link in links:
        print(f"{link}")
        try:
            all_reviews[link] = scrape_reviews(link)
        except Exception as e:
            all_reviews[link] = f"Error occurred: {str(e)}"
    return all_reviews

if __name__ == "__main__":
    business_name = input("Enter the business name: ")
    business_type = input("Enter business type (hotel, restaurant, or general): ").lower()
    reviews_data = get_business_reviews(business_name, business_type)

    with open(f"{business_name}_reviews.txt", "w", encoding="utf-8") as file:
        file.write(f"# {business_name} Reviews\n\n")
        for link, reviews in reviews_data.items():
            file.write(f"## Reviews from {link}:\n")
            if isinstance(reviews, list):
                file.write("\n".join(f"- {review}" for review in reviews) + "\n")  # Use join for efficiency
            else:
                file.write(f"{reviews}\n")
            file.write("\n")

    print(f"Reviews saved to {business_name}_reviews.txt")

Enter the business name: taro udaipur
Enter business type (hotel, restaurant, or general): restaurant
Request failed for query 'taro udaipur ' with status code: 429
Request failed for query 'taro udaipur ' with status code: 429
Request failed for query 'taro udaipur ' with status code: 429
Request failed for query 'taro udaipur ' with status code: 429
Request failed for query 'taro udaipur ' with status code: 429
Request failed for query 'taro udaipur reviews' with status code: 429
Request failed for query 'taro udaipur reviews' with status code: 429
Request failed for query 'taro udaipur reviews' with status code: 429
Request failed for query 'taro udaipur reviews' with status code: 429


KeyboardInterrupt: 